In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

In [2]:
# 3 states for the algorithm:
#    0. start with empty branch in which we will append leaves, we always try to append 1
#    1. NEW LEAF
#    2. END OF BRANCHÇ
#    3. AFTER PRUNE OR END OF BRANCH

In [3]:
#c = 9                     # knapsack capacity
#p = [6, 5, 8, 9, 6, 7, 3] # item profits
#w = [2, 3, 6, 7, 5, 9, 4] # item weights

c = 12
p = [12, 12, 11, 6, 2,10,8]
w = [4, 5, 3, 3, 2,1,2]

In [7]:
#returns solution of KP relaxed problem
def relax_kp(c, p, w, LogToConsole=False, TimeLimit=60):
    n = len(p)
    assert n == len(w), 'Lengths of p and w should be the same'

    model = Model()
    model.Params.LogToConsole = LogToConsole
    
    model.Params.Cuts = 0
    model.Params.Heuristics = 0
    model.Params.Presolve = 0

    x = model.addVars(n)

    model.setObjective(quicksum(p[i] * x[i] for i in range(n)), GRB.MAXIMIZE)

    model.addConstrs(x[i] >= 0 for i in range(n))
    model.addConstrs(x[i] <= 1 for i in range(n))

    model.addConstr(quicksum(w[i] * x[i] for i in range(n)) <= c)

    model.optimize()
    
    obj_list = []
    
    for v in model.getVars():
        obj_list.append((str(v.varName), float(v.x)))
        
    #return obj_list, model.objVal #in case Xs for lp solution are needed
    return model.objVal

In [5]:
#returns objects out of the kp and profit of present kp
def left_objects(branch,p,w):
    k = 0
    profit_now = 0
    c_left = c

    for item in branch:
        profit_now += item*p[k]
        c_left -= item*w[k]
        k += 1

    w_left = []
    p_left = []
    for j in range(len(branch),len(w)):
        w_left.append(w[j])
        p_left.append(p[j])
    
    return c_left, p_left, w_left, profit_now

In [15]:
#branch and cut algorithm
#variables c, p, w

n = len(p)
branch=[]
i=0
lb=0
best_branch=[]

print("### Knapsack Problem ###\n")
print("Capacity:\t",c,"\nPrices:\t\t", p,"\nWeights:\t", w)
print("\n### Start Branch & Cut ###\n")

algo_on = True
iter_count = 0
iter_max = 10000

while (algo_on):
    
    iter_count += 1
    if iter_count > iter_max: break
    
    #print("step: i=",i,"p=",p[i],"w=",w[i], "branch:", branch)
    if i > 0: print(branch)
    
    #END OF BRANCH
    if i == n-1:
        
        #if we can add one last object we add it
        proposal_c = c - sum([x*y for x,y in zip(branch+[1],w)])
        if not (proposal_c < 0): branch.append(1)
        
        #we calculate lower bound and update if better
        new_lb = sum([x*y for x,y in zip(branch,p)])
        
        #lb = max(lb, new_lb)
        if (new_lb > lb):
            print("  New best LB found:", new_lb,"\n  Best LB branch:", branch)
            lb = new_lb
            best_branch = branch.copy()
        
        #we set i to the last object that was chosen (last item branch[i] = 1) 
        non_zeros = [k for k, e in enumerate(branch) if e != 0]
        
        #if non_zeros is null then we exit 
        #it may not be necessary here as we never end branches with all 0s, we would have pruned before
        if not non_zeros: break
        
        #we don't need to explore the last item as 0 if it had been picked
        if max(non_zeros) == i: non_zeros.pop()
        
        del branch[max(non_zeros)+1:len(branch)]
        i = max(non_zeros)
        
        continue #branch finished, we go up the branch to find an object to unpick
    
    #NEW LEAF
    if i > len(branch) - 1: #we are out of bounds, we add new leaf
        
        proposal_c = c - sum([x*y for x,y in zip(branch+[1],w)])
        if proposal_c < 0: 
            #if new object does not fit, dont add it and continue to next object
            branch.append(0)
            i += 1
            continue
        else:
           #if it fits we pick up the object
            branch.append(1)
            
            #we build KP-relax model to obtain UB with unused objects
            c_left, p_left, w_left, profit_now = left_objects(branch,p,w)
            
            if relax_kp(c_left, p_left, w_left) + profit_now < lb:
                print("  UB < LB Pruning...")
                continue #prune
            else:
                i+=1
                continue #continue exploration

    #AFTER END OF BRANCH OR CUTTING
    else:
        #if sum of partial branch is 0 then we finished, best lb is solution
        if sum(branch)==0: break
        
        #we explore by changing the value of the last leaf to 0
        branch[i] = 0
        
        #calculate UB and check if it's lower than best LB. If it's the case cut, if not continue
        c_left, p_left, w_left, profit_now = left_objects(branch,p,w)

        if relax_kp(c_left, p_left, w_left) + profit_now < lb:
            
            non_zeros = [k for k, e in enumerate(branch) if e != 0]
            
            if not non_zeros: break
            if max(non_zeros) == i: non_zeros.pop()

            del branch[max(non_zeros)+1:len(branch)]
            i = max(non_zeros)
            
            print("  UB < LB Pruning...")
            continue #branch
        else:
            i+=1
            continue #continue exploration
    
    #something is wrong, we stop
    algo_on = False
    
print("\n### Final solution:",lb,"###")  
print("### Solution branch:",best_branch,"###")
    

### Knapsack Problem ###

Capacity:	 12 
Prices:		 [12, 12, 11, 6, 2, 10, 8] 
Weights:	 [4, 5, 3, 3, 2, 1, 2]

### Start Branch & Cut ###

[1]
[1, 1]
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0]
  New best LB found: 35 
  Best LB branch: [1, 1, 1, 0, 0, 0]
[1, 1, 1]
[1, 1, 0]
  UB < LB Pruning...
[1, 1, 0, 1]
[1, 1, 0, 0]
[1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
  New best LB found: 36 
  Best LB branch: [1, 1, 0, 0, 1, 1]
[1, 1, 0, 0, 1, 1]
  UB < LB Pruning...
[1, 1, 0, 0, 1]
[1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1]
  New best LB found: 42 
  Best LB branch: [1, 1, 0, 0, 0, 1, 1]
[1, 1, 0, 0, 0, 1]
  UB < LB Pruning...
[1, 1]
[1, 0]
[1, 0, 1]
[1, 0, 1, 1]
  UB < LB Pruning...
[1, 0, 1, 1, 1]
[1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 1]
  UB < LB Pruning...
[1, 0, 1, 1]
[1, 0, 1, 0]
[1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 1]
  New best LB found: 43 
  Best LB branch: [1, 0, 1, 0, 1, 1, 1]
[1, 0, 1, 0, 1, 1]
  UB < LB Pruning...
[1, 0, 1, 0, 1]
  UB < LB Pruning...
[1, 0, 1]
  UB < L